In [1]:
pip install requests beautifulsoup4 pandas

You should consider upgrading via the 'c:\Users\eeyy1\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
pip install selenium webdriver-manager

  Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl (27 kB)
  Using cached python_dotenv-1.2.1-py3-none-any.whl (21 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\eeyy1\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [24]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# 1. 타겟 URL 및 헤더 설정
url = "https://finance.naver.com/research/debenture_list.naver"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
    'Referer': 'https://finance.naver.com/research/debenture_list.naver'
}

try:
    # 2. 페이지 요청
    res = requests.get(url, headers=headers)
    res.encoding = 'euc-kr'
    soup = BeautifulSoup(res.text, 'html.parser')
    
    # 3. 데이터 추출 (강사님 피드백: 클래스명을 'type_1'로 수정)
    # 강사님이 이미지에서 짚어주신대로 table.type_1 내부의 tr을 찾습니다.
    rows = soup.select('table.type_1 tr')
    
    bond_data = []
    
    for row in rows:
        tds = row.find_all('td')
        # 데이터가 있는 행은 보통 td가 5개입니다.
        if len(tds) >= 4:
            company = tds[0].text.strip() # 증권사
            title_tag = tds[1].find('a')  # 제목 태그
            
            if title_tag:
                title = title_tag.text.strip()
                # 상세 페이지 링크 (PDF를 따기 위해 필요)
                link = "https://finance.naver.com/research/" + title_tag['href']
                date = tds[4].text.strip() # 작성일
                
                bond_data.append([date, company, title, link])
                print(f"[성공] {date} | {company} | {title[:15]}...")

    # 4. 결과 확인
    df = pd.DataFrame(bond_data, columns=['Date', 'Company', 'Title', 'Link'])
    
    print("\n" + "="*50)
    print(f"최종 추출 건수: {len(df)}건")
    print("="*50)
    
    if not df.empty:
        print(df.head(10))
    else:
        print("여전히 데이터가 0개라면 클래스명 외에 다른 구조도 확인이 필요합니다.")

except Exception as e:
    print(f"에러 발생: {e}")


최종 추출 건수: 0건
여전히 데이터가 0개라면 클래스명 외에 다른 구조도 확인이 필요합니다.


In [25]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 1. 세션(Session)을 생성하여 쿠키를 자동으로 관리하게 합니다.
session = requests.Session()

url = "https://finance.naver.com/research/debenture_list.naver"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Referer': 'https://finance.naver.com/research/debenture_list.naver',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8'
}

try:
    # 세션을 통해 첫 접속을 시도하여 쿠키를 굽습니다.
    res = session.get(url, headers=headers)
    res.encoding = 'euc-kr'
    
    soup = BeautifulSoup(res.text, 'html.parser')
    
    # 강사님이 짚어준 필살 셀렉터
    rows = soup.select('table.type_1 tr')
    
    bond_data = []
    for row in rows:
        tds = row.find_all('td')
        # 네이버 채권 리스트는 td가 보통 6개입니다. (번호, 제목, 증권사, 조회수, 날짜 등)
        if len(tds) >= 4:
            # 강사님 이미지 구조대로 인덱스를 재조정합니다.
            title_tag = tds[0].find('a') # 보통 첫 번째나 두 번째에 제목이 있습니다.
            if title_tag:
                title = title_tag.text.strip()
                company = tds[1].text.strip()
                date = tds[4].text.strip()
                link = "https://finance.naver.com/research/" + title_tag['href']
                bond_data.append([date, company, title, link])

    df = pd.DataFrame(bond_data)
    print(f"--- 추출 결과: {len(df)}건 ---")
    if not df.empty:
        print(df.head())

except Exception as e:
    print(f"에러: {e}")

--- 추출 결과: 30건 ---
    0       1                                     2  \
0  77    하나증권          성장률 컨센서스와 금리(’14, ’17, ’26년)   
1  52  다올투자증권  Daily Bond Morning Brief(2025.12.30)   
2  42  다올투자증권  Daily Bond Morning Brief(2025.12.29)   
3  55   유안타증권                      채권 Daily (12.30)   
4  50  유진투자증권                   Eugenes FICC Update   

                                                   3  
0  https://finance.naver.com/research/debenture_r...  
1  https://finance.naver.com/research/debenture_r...  
2  https://finance.naver.com/research/debenture_r...  
3  https://finance.naver.com/research/debenture_r...  
4  https://finance.naver.com/research/debenture_r...  


In [27]:
import os
import urllib.request
import re

# 1. 저장 폴더 설정
save_dir = "bond_reports"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

print(f"총 {len(bond_data)}개의 PDF를 정제 후 다운로드합니다...")

for i, report in enumerate(bond_data):
    # report 구조: [0:조회수, 1:증권사, 2:제목, 3:상세링크]
    # (주의: 아까 결과에서 컬럼 순서가 다를 수 있으니 인덱스 확인!)
    
    title_raw = report[2] 
    company = report[1]
    detail_url = report[3]
    
    try:
        # 상세 페이지 접속
        res = session.get(detail_url, headers=headers)
        res.encoding = 'euc-kr'
        detail_soup = BeautifulSoup(res.text, 'html.parser')
        
        pdf_btn = detail_soup.select_one('a.file_down')
        if pdf_btn:
            pdf_url = pdf_btn['href']
            
            # [오류 해결 포인트] 제목 내 특수문자('14, '17 등)를 안전한 문자로 변경
            # 파일명에 쓸 수 없는 문자나 홑따옴표를 언더바(_)나 공백으로 치환합니다.
            clean_title = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "_", title_raw)
            clean_title = clean_title.replace("__", "_").strip() # 연속된 언더바 정리
            
            file_name = f"{save_dir}/{company}_{clean_title[:40]}.pdf"
            
            # PDF 다운로드
            urllib.request.urlretrieve(pdf_url, file_name)
            print(f"[{i+1}] 저장 성공: {file_name}")
            
            time.sleep(0.5)
            
    except Exception as e:
        print(f"[{i+1}] 실패: {title_raw[:20]}... (에러: {e})")

print("\n--- 모든 오류를 잡고 다운로드를 완료했습니다! ---")

print(df.head())

총 30개의 PDF를 정제 후 다운로드합니다...

--- 모든 오류를 잡고 다운로드를 완료했습니다! ---
    0       1                                     2  \
0  77    하나증권          성장률 컨센서스와 금리(’14, ’17, ’26년)   
1  52  다올투자증권  Daily Bond Morning Brief(2025.12.30)   
2  42  다올투자증권  Daily Bond Morning Brief(2025.12.29)   
3  55   유안타증권                      채권 Daily (12.30)   
4  50  유진투자증권                   Eugenes FICC Update   

                                                   3  
0  https://finance.naver.com/research/debenture_r...  
1  https://finance.naver.com/research/debenture_r...  
2  https://finance.naver.com/research/debenture_r...  
3  https://finance.naver.com/research/debenture_r...  
4  https://finance.naver.com/research/debenture_r...  


In [28]:
import os
import re
import requests

# 1. 저장 폴더 다시 확인
save_dir = "bond_reports_final"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

print(f"총 {len(bond_data)}개의 리포트 분석 및 다운로드 시작...")

for i, report in enumerate(bond_data):
    # 데이터 구조에 따라 인덱스를 조정하세요 (현재: 1=증권사, 2=제목, 3=상세링크)
    company = report[1]
    title_raw = report[2]
    detail_url = report[3]
    
    try:
        # A. 상세 페이지 접속 (PDF 주소를 찾기 위함)
        res = session.get(detail_url, headers=headers)
        res.encoding = 'euc-kr'
        soup = BeautifulSoup(res.text, 'html.parser')
        
        # B. PDF 주소 추출
        pdf_btn = soup.select_one('a.file_down')
        if not pdf_btn:
            print(f"[{i}] PDF 링크를 찾을 수 없음: {title_raw[:15]}")
            continue
            
        pdf_url = pdf_btn['href']
        
        # C. [핵심] 파일명 정제 (특수문자 '14, '17 등을 언더바 또는 제거)
        # 윈도우 파일명 금지 문자: \ / : * ? " < > | 및 따옴표 제거
        clean_title = re.sub(r'[\/:*?"<>|]', '', title_raw) # 금지 문자 제거
        clean_title = clean_title.replace("’", "").replace("'", "") # 따옴표 제거
        file_name = f"{company}_{clean_title[:30]}.pdf"
        file_path = os.path.join(save_dir, file_name)
        
        # D. [핵심] PDF 파일 다운로드 (Streaming 방식)
        # 단순히 주소를 요청하는 게 아니라 파일 데이터를 쪼개서 받아옵니다.
        pdf_res = session.get(pdf_url, headers=headers, stream=True)
        
        if pdf_res.status_code == 200:
            with open(file_path, 'wb') as f:
                for chunk in pdf_res.iter_content(chunk_size=1024):
                    if chunk:
                        f.write(chunk)
            print(f"[{i+1}] 성공: {file_name}")
        else:
            print(f"[{i+1}] 실패: 서버 응답 코드 {pdf_res.status_code}")
            
    except Exception as e:
        print(f"[{i+1}] 에러 발생: {str(e)}")

print("\n--- 모든 시도가 완료되었습니다. 폴더를 확인하세요! ---")

print(df.head())


총 30개의 리포트 분석 및 다운로드 시작...
[0] PDF 링크를 찾을 수 없음: 성장률 컨센서스와 금리(’1
[1] PDF 링크를 찾을 수 없음: Daily Bond Morn
[2] PDF 링크를 찾을 수 없음: Daily Bond Morn
[3] PDF 링크를 찾을 수 없음: 채권 Daily (12.30
[4] PDF 링크를 찾을 수 없음: Eugenes FICC Up
[5] PDF 링크를 찾을 수 없음: [FI Monthly] Ca
[6] PDF 링크를 찾을 수 없음: KP: 1월, 쏟아지는 공급
[7] PDF 링크를 찾을 수 없음: 채권 Daily (12.29
[8] PDF 링크를 찾을 수 없음: Eugenes FICC Up
[9] PDF 링크를 찾을 수 없음: 환율은 진정되는 듯하지만..
[10] PDF 링크를 찾을 수 없음: 채권 Daily (12.26
[11] PDF 링크를 찾을 수 없음: Daily Bond Morn
[12] PDF 링크를 찾을 수 없음: Daily Bond Morn
[13] PDF 링크를 찾을 수 없음: Eugenes FICC Up
[14] PDF 링크를 찾을 수 없음: 최근 신용스프레드 하방경직성
[15] PDF 링크를 찾을 수 없음: 채권 Daily (12.24
[16] PDF 링크를 찾을 수 없음: Eugenes FICC Up
[17] PDF 링크를 찾을 수 없음: 채권 Daily (12.23
[18] PDF 링크를 찾을 수 없음: Daily Bond Morn
[19] PDF 링크를 찾을 수 없음: Eugenes FICC Up
[20] PDF 링크를 찾을 수 없음: Weekly Credit M
[21] PDF 링크를 찾을 수 없음: [FI Weekly] 상반기
[22] PDF 링크를 찾을 수 없음: [12월 BOJ] 중립금리를
[23] PDF 링크를 찾을 수 없음: 채권시장이 바라는 연준 의장
[24] PDF 링크를 찾을 수 없음: 채권 Daily (12.22
[25] PDF 링크를 찾을 수 없음: 12/22 Kiwoo

In [29]:
import os
import re
import requests
from bs4 import BeautifulSoup

save_dir = "bond_reports_final"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

print("성벽을 넘기 위해 전략을 바꿨습니다. PDF 주소 정밀 추적 시작...")

for i, report in enumerate(bond_data):
    company = report[1]
    title_raw = report[2]
    detail_url = report[3]
    
    try:
        res = session.get(detail_url, headers=headers)
        res.encoding = 'euc-kr'
        soup = BeautifulSoup(res.text, 'html.parser')
        
        # [핵심 변경] 특정 클래스명이 아니라, href 속성에 '.pdf'가 포함된 모든 a 태그를 찾습니다.
        pdf_link = None
        all_links = soup.find_all('a', href=True)
        for link in all_links:
            if '.pdf' in link['href'].lower():
                pdf_link = link['href']
                break
        
        if not pdf_link:
            # 만약 a 태그에 없다면, 버튼(button) 태그 내의 onclick 자바스크립트 등도 확인해봐야 하지만
            # 우선 가장 확률 높은 방식으로 시도합니다.
            print(f"[{i}] 여전히 링크 못 찾음: {title_raw[:15]}")
            continue
            
        # 파일명 정제 (특수문자 제거)
        clean_title = re.sub(r'[^\uAC00-\uD7A30-9a-zA-Z\s]', '', title_raw)
        file_name = f"{company}_{clean_title[:20]}.pdf"
        file_path = os.path.join(save_dir, file_name)
        
        # 다운로드 시도
        pdf_res = session.get(pdf_link, headers=headers, stream=True)
        if pdf_res.status_code == 200:
            with open(file_path, 'wb') as f:
                for chunk in pdf_res.iter_content(chunk_size=1024):
                    f.write(chunk)
            print(f"[{i+1}] 드디어 성공!!: {file_name}")
        else:
            print(f"[{i+1}] 서버 응답 에러: {pdf_res.status_code}")
            
    except Exception as e:
        print(f"[{i+1}] 에러: {str(e)}")

print("\n작업 완료. 폴더에 파일이 하나라도 생겼는지 확인해주세요!")

성벽을 넘기 위해 전략을 바꿨습니다. PDF 주소 정밀 추적 시작...
[1] 드디어 성공!!: 하나증권_성장률 컨센서스와 금리14 17 26.pdf
[2] 드디어 성공!!: 다올투자증권_Daily Bond Morning B.pdf
[3] 드디어 성공!!: 다올투자증권_Daily Bond Morning B.pdf
[4] 드디어 성공!!: 유안타증권_채권 Daily 1230.pdf
[5] 드디어 성공!!: 유진투자증권_Eugenes FICC Update.pdf
[6] 드디어 성공!!: 한화투자증권_FI Monthly Carry New.pdf
[7] 드디어 성공!!: 하나증권_KP 1월 쏟아지는 공급과 기회의 창.pdf
[8] 드디어 성공!!: 유안타증권_채권 Daily 1229.pdf
[9] 드디어 성공!!: 유진투자증권_Eugenes FICC Update.pdf
[9] 여전히 링크 못 찾음: 환율은 진정되는 듯하지만..
[11] 드디어 성공!!: 유안타증권_채권 Daily 1226.pdf
[12] 드디어 성공!!: 다올투자증권_Daily Bond Morning B.pdf
[13] 드디어 성공!!: 다올투자증권_Daily Bond Morning B.pdf
[14] 드디어 성공!!: 유진투자증권_Eugenes FICC Update.pdf
[15] 드디어 성공!!: iM증권_최근 신용스프레드 하방경직성과 거래대.pdf
[16] 드디어 성공!!: 유안타증권_채권 Daily 1224.pdf
[17] 드디어 성공!!: 유진투자증권_Eugenes FICC Update.pdf
[18] 드디어 성공!!: 유안타증권_채권 Daily 1223.pdf
[19] 드디어 성공!!: 다올투자증권_Daily Bond Morning B.pdf
[20] 드디어 성공!!: 유진투자증권_Eugenes FICC Update.pdf
[20] 여전히 링크 못 찾음: Weekly Credit M
[22] 드디어 성공!!: 한화투자증권_FI Weekly 상반기 수급부담 감.pdf
[23] 드디어 성공!!

In [30]:
import time
import pandas as pd

# 수집할 총 리포트 목록을 담을 리스트
all_bond_data = []

# 시작 페이지와 끝 페이지 설정 (2005년까지 가려면 수백 페이지가 필요할 수 있습니다)
# 우선 1페이지부터 100페이지까지 시도해보고, 더 필요하면 숫자를 늘리세요.
start_page = 1
end_page = 500 # 네이버 채권 끝 페이지까지 확인 필요

print(f"{start_page}페이지부터 수집을 시작합니다. 대장정의 시작!")

for page in range(start_page, end_page + 1):
    page_url = f"https://finance.naver.com/research/debenture_list.naver?page={page}"
    
    try:
        # 1. 페이지 요청
        res = session.get(page_url, headers=headers)
        res.encoding = 'euc-kr'
        soup = BeautifulSoup(res.text, 'html.parser')
        
        # 2. 리스트 추출 (강사님이 짚어준 type_1)
        rows = soup.select('table.type_1 tr')
        
        # 이번 페이지에서 새로 찾은 데이터 개수 확인
        current_page_count = 0
        for row in rows:
            tds = row.find_all('td')
            if len(tds) >= 4:
                title_tag = tds[0].find('a')
                if title_tag:
                    title = title_tag.text.strip()
                    company = tds[1].text.strip()
                    date = tds[4].text.strip()
                    link = "https://finance.naver.com/research/" + title_tag['href']
                    
                    all_bond_data.append([date, company, title, link])
                    current_page_count += 1
        
        print(f"[Page {page}] {current_page_count}건 수집 완료 (누적: {len(all_bond_data)}건)")
        
        # 3. 브레이크 포인트 (데이터가 없는 페이지에 도달하면 멈춤)
        if current_page_count == 0:
            print("더 이상 데이터가 없는 페이지입니다. 수집을 종료합니다.")
            break
            
        # 4. 서버 과부하 방지 (중요!)
        # 너무 빨리 요청하면 IP 차단당하니 0.3초 정도 쉬어줍니다.
        time.sleep(0.3)
        
    except Exception as e:
        print(f"[Page {page}] 에러 발생: {e}")
        continue

# 전체 목록 저장
final_df = pd.DataFrame(all_bond_data, columns=['날짜', '증권사', '제목', '링크'])
final_df.to_csv("naver_bond_all_list.csv", index=False, encoding='utf-8-sig')
print("전체 리스트가 naver_bond_all_list.csv로 저장되었습니다.")

1페이지부터 수집을 시작합니다. 대장정의 시작!
[Page 1] 30건 수집 완료 (누적: 30건)
[Page 2] 30건 수집 완료 (누적: 60건)
[Page 3] 30건 수집 완료 (누적: 90건)
[Page 4] 30건 수집 완료 (누적: 120건)
[Page 5] 30건 수집 완료 (누적: 150건)
[Page 6] 30건 수집 완료 (누적: 180건)
[Page 7] 30건 수집 완료 (누적: 210건)
[Page 8] 30건 수집 완료 (누적: 240건)
[Page 9] 30건 수집 완료 (누적: 270건)
[Page 10] 30건 수집 완료 (누적: 300건)
[Page 11] 30건 수집 완료 (누적: 330건)
[Page 12] 30건 수집 완료 (누적: 360건)
[Page 13] 30건 수집 완료 (누적: 390건)
[Page 14] 30건 수집 완료 (누적: 420건)
[Page 15] 30건 수집 완료 (누적: 450건)
[Page 16] 30건 수집 완료 (누적: 480건)
[Page 17] 30건 수집 완료 (누적: 510건)
[Page 18] 30건 수집 완료 (누적: 540건)
[Page 19] 30건 수집 완료 (누적: 570건)
[Page 20] 30건 수집 완료 (누적: 600건)
[Page 21] 30건 수집 완료 (누적: 630건)
[Page 22] 30건 수집 완료 (누적: 660건)
[Page 23] 30건 수집 완료 (누적: 690건)
[Page 24] 30건 수집 완료 (누적: 720건)
[Page 25] 30건 수집 완료 (누적: 750건)
[Page 26] 30건 수집 완료 (누적: 780건)
[Page 27] 30건 수집 완료 (누적: 810건)
[Page 28] 30건 수집 완료 (누적: 840건)
[Page 29] 30건 수집 완료 (누적: 870건)
[Page 30] 30건 수집 완료 (누적: 900건)
[Page 31] 30건 수집 완료 (누적: 930건)
[Page 32] 30건 수집 완료 (누적:

In [ ]:
import os
import re
import time
import random
import requests
import pdfplumber
import io
import logging
from bs4 import BeautifulSoup

# [1] 지저분한 경고 메시지(WARNING) 출력 차단
logging.getLogger("pdfminer").setLevel(logging.ERROR)

# [2] 저장 경로 설정 (v5로 새로 시작)
save_dir = '/content/drive/MyDrive/bond_text_final_v5'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

session = requests.Session()
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0'}

page = 1
target_date = "05.05.01"
stop_collecting = False

print("🚀 수집 시작! 이제 날짜가 정확한 것만 화면에 보입니다.")

while not stop_collecting and page < 2000:
    url = f"https://finance.naver.com/research/debenture_list.naver?page={page}"
    try:
        res = session.get(url, headers=headers)
        res.encoding = 'euc-kr'
        soup = BeautifulSoup(res.text, 'html.parser')
        
        # [3] 날짜가 포함된 td를 더 정확하게 타겟팅 (class가 'date'인 것 찾기)
        rows = soup.select('table.type_1 tr')
        
        found_on_page = 0
        for row in rows:
            # 날짜 칸(보통 마지막 td이거나 class="date"인 td)을 찾습니다.
            date_td = row.select_one('td.date')
            if not date_td:
                # class가 없다면 모든 td 중 날짜 형식인 것을 찾음
                tds = row.find_all('td')
                if not tds: continue
                raw_date = tds[-1].text.strip()
            else:
                raw_date = date_td.text.strip()

            # [4] 정규식으로 '24.12.30' 같은 형식인지 엄격하게 검사
            if not re.match(r'^\d{2}\.\d{2}\.\d{2}$', raw_date):
                continue # 20844 같은 숫자는 여기서 무조건 걸러집니다.

            if raw_date < target_date:
                stop_collecting = True
                break
            
            # 날짜 변환 (예: 20251230)
            clean_date = "20" + raw_date.replace(".", "")
            
            # 증권사 및 제목 추출
            tds = row.find_all('td')
            company = tds[1].text.strip().replace(" ", "")
            title_tag = tds[0].find('a')
            if not title_tag: continue
            
            title = title_tag.text.strip()
            detail_url = "https://finance.naver.com/research/" + title_tag['href']
            
            # 파일명 생성 (공백 없이 날짜가 맨 앞으로!)
            safe_title = re.sub(r'[^\uAC00-\uD7A30-9a-zA-Z]', '', title)
            file_name = f"{clean_date}_{company}_{safe_title[:20]}.txt"
            file_path = os.path.join(save_dir, file_name)
            
            if os.path.exists(file_path): continue

            try:
                # PDF 추출
                d_res = session.get(detail_url, headers=headers)
                d_res.encoding = 'euc-kr'
                pdf_url = None
                d_soup = BeautifulSoup(d_res.text, 'html.parser')
                for a in d_soup.find_all('a', href=True):
                    if '.pdf' in a['href'].lower():
                        pdf_url = a['href']
                        break
                
                if pdf_url:
                    p_res = session.get(pdf_url, headers=headers)
                    with pdfplumber.open(io.BytesIO(p_res.content)) as pdf:
                        full_text = "\n".join([p.extract_text() for p in pdf.pages if p.extract_text()])
                    
                    if full_text.strip():
                        with open(file_path, 'w', encoding='utf-8') as f:
                            f.write(full_text)
                        print(f"✅ [저장성공] {file_name}") # 이 메시지만 보일 겁니다!
                        found_on_page += 1
                
                time.sleep(random.uniform(0.3, 0.5))
            except:
                continue

        if found_on_page > 0:
            print(f"--- {page}페이지 작업 완료 ---")
        
        page += 1
        # 네이버가 1000페이지 넘어가면 데이터를 안 줄 때를 대비
        if found_on_page == 0 and page > 100:
            # 5페이지 연속으로 데이터를 못 찾으면 진짜 끝난 것으로 간주
            empty_check_count = vars().get('empty_check_count', 0) + 1
            if empty_check_count > 5: break
        else:
            empty_check_count = 0

    except Exception as e:
        print(f"🌐 접속 에러: {e}")
        break

print("🎉 수집이 완료되었습니다! 드라이브 v5 폴더를 확인하세요.")